# 지도학습-로지스틱|성능평가함수

In [2]:
from hossam import *

# 파이썬 기본 라이브러리
from pandas import DataFrame, concat, get_dummies
import numpy as np
from itertools import combinations

# 시각화 라이브러리
from matplotlib import pyplot as plt
import seaborn as sb

# 통계 라이브러리
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import linear_reset, het_breuschpagan, het_white
from statsmodels.stats.stattools import durbin_watson
from pingouin import anova, welch_anova
from scipy.stats import (
    normaltest,
    bartlett,
    levene,
    ttest_ind,
    pearsonr,
    spearmanr,
    shapiro,
    jarque_bera,
    t,
    f,
)

# 머신러닝 라이브러리
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.impute import SimpleImputer

# 학습모델
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    SGDRegressor
)

# 성능 평가 지표 모듈
from sklearn.metrics import(
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    fl_score,
    roc_curve,
    roc_auc_score
)

# 확률적 경사 하강법
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance

# SVR
from sklearn.svm import SVR
from sklearn.inspection import permutation_importance

# 로지스틱 회귀 모형
from sklearn.linear_model import LogisticRegression

# 로지스틱 성능평가 함수
from sklearn.metrics import (confusion_matrix, roc_curve, roc_auc_score, accuracy_score, recall_score, 
                            precision_score, f1_score, precision_recall_curve, log_loss)

# 군집 시각화 참조
from scipy.spatial import ConvexHull

📦 아이티윌 이광호 강사가 제작한 라이브러리를 사용중입니다.
📚 자세한 사용 방법은 https://py.hossam.kr 을 참고하세요.
📧 Email: leekh4232@gmail.com
🎬 Youtube: https://www.youtube.com/@hossam-codingclub
📝 Blog: https://blog.hossam.kr/
🔖 Version: 0.5.3

✅ 시각화를 위한 한글 글꼴(NotoSansKR-Regular)이 자동 적용되었습니다.


ImportError: cannot import name 'fl_score' from 'sklearn.metrics' (c:\Users\wodyd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\__init__.py)

In [ ]:
origin = load_data('pima_indians_diabetes_preprocessed')
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print(origin.info())
origin.head()

캐글에서 제공하는 pima_indians_diabetes의 전처리 완료 버전(결측치 정제+로그변환) (출처: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)
데이터셋 크기: (768, 9)
열 개수: 9
행 개수: 768
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    float64
 1   Glucose                   768 non-null    float64
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    float64
 4   Insulin                   768 non-null    float64
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    float64
 8   Outcome                   768 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 54.1 KB
None


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1.946,5.004,72,3.584,4.836,3.544,0.487,3.932,1
1,0.693,4.454,66,3.401,4.836,3.318,0.301,3.466,0
2,2.197,5.215,64,3.401,4.836,3.190,0.514,3.497,1
3,0.693,4.500,66,3.178,4.554,3.371,0.154,3.091,0
4,0.000,4.927,40,3.584,5.130,3.786,1.190,3.526,1


## 1. 준비작업
### 1. 훈련 - 검증 데이터 분리

In [ ]:
df = origin.copy()

# 종속변수 -> 정수형으로 변환
df['Outcome'] = df['Outcome'].astype('int')
yname = 'Outcome'
x = df.drop(columns = [yname])
y = df[yname]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.2, random_state = 52
)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

## 2. 로지스틱 모형 적합

In [ ]:
%%time

pipe = Pipeline([('VIF_Selector', VIFSelector()), ('scaler', StandardScaler()), ('model', LogisticRegression(random_state = 52))])

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    'model__penalty': ['l2'],
    'model__solver': ['lbfgs'],
    'model__C': [0.01, 0.1, 1, 10, 100],
    'model__max_iter': [100, 300, 500],
    'model__class_weight': [None, 'balanced']
    
    # poly 커널 확장용 (필요시 사용, 학습속도가 매우 느려짐)
    # 'model__kernel':['poly'],
    # 'model__degree':[2, 3, 4],
    # 'model__coef0':[0.0, 0.5, 1.0]
    }

# 그리드 서치 객체 생성
gs = GridSearchCV(estimator = pipe, param_grid = param_grid, cv=5, scoring ='roc_auc', n_jobs = -1)

gs.fit(x_train, y_train)
estimator = gs.best_estimator_
estimator

CPU times: total: 406 ms
Wall time: 26.4 s


Pipeline(steps=[('VIF_Selector', VIFSelector()), ('scaler', StandardScaler()),
                ('model',
                 LogisticRegression(C=1, class_weight='balanced',
                                    random_state=52))])

## 3. 성능평가
### 1. 성능평가 함수
#### 1. 함수 정의

In [ ]:
def hs_cls_bin_scores(estimator, x_test, y_test):
    # 위, 양성 확률
    y_pred_proba = estimator.predict_proba(x_test)
    # 1로 분류될 확률
    y_pred_proba_1 = estimator.predict_proba(x_test)[:,1]
    # 예측값
    y_pred = estimator.predict(x_test)

    # 의사결정계수
    log_loss_test = -log_loss(y_test, y_pred_proba, normalize = False)
    y_null = np.ones_like(y_test) * y_test.mean()
    log_loss_null = -log_loss(y_test, y_null, normalize = False)
    pseudo_r2 = 1 - (log_loss_test / log_loss_null)

    # 혼동행렬
    cm = confusion_matrix(y_test, y_pred)
    ((TN, FP), (FN, TP)) = cm

    # 클래스 이름
    if hasattr(estimator, 'named_steps'):
        classname = estimator.named_steps['model'].__class__.__name__
    else:
        classname = estimator.__class__.__name__

    # auc score
    auc = roc_auc_score(y_test, y_pred_proba_1)

    score_df = DataFrame({
        '정확도(Accuracy)': [accuracy_score(y_test, y_pred)],
        '정밀도(Precision)': [precision_score(y_test, y_pred)],
        '재현율(Recall, tpr)': [recall_score(y_test, y_pred)],
        '위양성율(Fallout,fpr)': [FP / (TN + FP)],
        '특이성(TNR)': [1 - (FP / (TN + FP))],
        'F1 Score': [fl_score(y_test, y_pred)],
        'AUC': [auc]
    }, index = [classname]
    )

    auc = roc_auc_score(y_test, y_pred_proba_1)

    roc_fpr, roc_tpr, thresholds = roc_curve(y_test, y_pred_proba_1)

    width_px = 1000
    height_px = 480
    rows = 1
    cols = 1
    figsize = (width_px/ my_dpi, height_px / my_dpi)
    fig, ax = plt.subplots(rows, cols, figsize = figsize, dpi = my_dpi)

    # 그래프 그리기, seaborn 사용
    sb.lineplot(x = roc_fpr, y = roc_tpr)

    sb.lineplot(x= [0, 1], y = [0, 1],
            color = 'red', linestyle = ':', alpha = 0.5
            )
    plt.fill_between(x=roc_fpr, y1 = roc_tpr, alpha = 0.1)
    # 그래프 꾸미기
    ax.grid(True) # 배경 격자 표시/숨김
    ax.set_title(f'AUC = {auc:.4f}', pad = 10, fontsize = 4)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    # 출력
    plt.grid()
    plt.tight_layout()
    plt.show()
    plt.close()

    return score_df

#### 2. 함수 확인


In [ ]:
hs_cls_bin_scores(estimator, x_test, y_test)

NameError: name 'fl_score' is not defined

### 2. 과적합 판정
#### 1. 함수 개선

In [4]:
def hs_learning_cv(
        estimator,
        x,
        y,
        scoring = None, # None으로 기본값 수정
        cv = 5,
        train_sizes = np.linspace(0.1, 1.0, 10),
        n_jobs = -1
):
    # 추가: 문제 유형 자동 판별
    is_classification = (
        hasattr(estimator, "_estimator_type")
        and estimator._estimaotr_type == 'classifier'
    )

    # scoring 자동 설정

    if scoring is None:
        scoring = 'roc_auc' if is_classification else 'neg_root_mean_squared_error'

    # learning curve 계산 (기존과 동일)
    train_sizes, train_scores, cv_scores = learning_curve(
    # type: ignore
    estimator = estimator,
    X=x,
    y=y,
    train_sizes = train_sizes,
    cv = cv,
    scoring = scoring,
    n_jobs = n_jobs,
    shuffle = False,
    random_state = 52
)

    # 모델명 추출(Pipeline 대응, 변경 여부 체크 필요)
    if hasattr(estimator, 'named_steps'):
        classname = estimator.named_steps['model'].__class__.__name__
    else:
        classname = estimator.__class__.__name__

    # 회귀 전용 처리(기존 코드, if문은 추가)
    if not is_classification:

        # neg RMSE -> RMSE
        train_rmse = -train_scores
        cv_rmse = -cv_scores

        # 평균 / 표준편차
        train_mean = train_rmse.mean(axis = 1)
        cv_mean = cv_rmse.mean(axis = 1)
        cv_std = cv_rmse.std(axis = 1)

        # 마지막 지점 기준 정량 판정
        final_train = train_mean[-1]
        final_cv = cv_mean[-1]
        final_std = cv_std[-1]
        gap_ratio = final_train / final_cv
        var_ratio = final_std / final_cv

        # 과소적합 기준선 (some_threshold)
        # 기준모형 RMSE(평균 예측)
        y_mean = y.mean()
        rmse_naive = np.sqrt(np.mean((y-y_mean) ** 2))

        # 분산 기반
        std_y = y.std()

        # 최소 설명력(R2) 기반
        min_r2 = 0.10
        rmse_r2 = np.sqrt((1 - min_r2) * np.var(y))

        # 최종 threshold (가장 관대한 기준)
        # -> 원래 some_threshold는 도메인 지식 수준에서 이 모델은 최소 어느 정도의 성능은 내야 한다는 기준을 설정하는 것
        some_threshold = min(rmse_naive, std_y, rmse_r2)

        # 판정 로직
        # 마지막 두 지점 기울기
        train_slope = train_mean[-1] - train_mean[-2]
        cv_slope = cv_mean[-1] - cv_mean[-2]

        # 분류 판정 로직(객관적 기준)
        if gap_ratio >= 0.95 and final_cv > some_threshold:
            status = '과소적합'
        elif gap_ratio <= 0.8 and train_slope > 0 and cv_slope < 0:
            status = '데이터 추가시 일반화 기대'
        elif gap_ratio <= 0.8:
            status = '과대적합'
        elif gap_ratio <= 0.95 and var_ratio <= 0.10:
            status = '일반화 양호'
        elif var_ratio > 0.15:
            status = '데이터 부족'
        else:
            status = '판단유보'

            # 추가: 평가 지표 이름
            metric_name = 'RMSE'

    # 추가: 분류 전용 처리
    else:
        # 분류는 그대로 사용 (AUC, Accuracy 등)
        train_metric = train_scores
        cv_metric = cv_scores

        train_mean = train_metric.mean(axis = 1)
        cv_mean = cv_metric.mean(axis = 1)
        cv_std = cv_metric.std(axis = 1)

        final_train = train_mean[-1]
        final_cv = cv_mean[-1]
        final_std = cv_std[-1]
        
        # 분류용 비율 정의 (차이 기반)
        gap_ratio = final_train - final_cv
        var_ratio = final_std

        # 분류 판정 로직 (객관적 기준)
        if final_train < 0.6 and final_cv < 0.6:
            status = '과소적합'
        elif gap_ratio > 0.1:
            status = '과대적합'
        elif gap_ratio <= 0.05 and var_ratio <= 0.05:
            status = '일반화 양호'
        elif var_ratio > 0.1:
            status = '데이터 부족'
        else:
            status = '판단유보'
        
        metirc_name = scoring.upper()
    result_df = DataFrame(
        {
        f'Train {metric_name}': [final_train],
        f'CV {metric_name}평균': [final_cv],
        f'CV {metric_name}표준편차':[final_std],
        f'Train/CV 비율':[gap_ratio],
        f'CV 변동성 비율':[var_ratio],
        '판정 결과': [status]
        },
        index = [classname]
    )
    width_px = 1600
    height_px = 620
    rows = 1
    cols = 1
    figsize = (width_px/ my_dpi, height_px / my_dpi)
    fig, ax = plt.subplots(rows, cols, figsize = figsize, dpi = my_dpi)

    # 그래프 그리기, seaborn 사용
    sb.lineplot(x = train_sizes, y = train_mean, marker = 'o', markeredgecolor = '#ffffff',
                label = f'Train { metric_name}')

    sb.lineplot(x = train_sizes, y = cv_mean, marker = 'o', markeredgecolor = '#ffffff',
                label = f'CV { metric_name}')
            
    # 그래프 꾸미기
    ax.grid(True) # 배경 격자 표시/숨김
    ax.set_xlabel('학습 데이터 비율')
    ax.set_ylabel(metric_name)
    # 출력
    plt.grid()
    plt.tight_layout()
    plt.show()
    plt.close()

    return result_df

#### 2. 함수 확인
